In [1]:
!pip install -q einops

## IMPORT LIBS

In [2]:
import os
import numpy as np
import random
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.amp import autocast
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torchvision.models as models
from tqdm import tqdm
import gc
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from einops import rearrange
import h5py
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
import seaborn as sns
from typing import List, Tuple
from datetime import datetime, timedelta
from pprint import pprint
import json
import math

## REPRODUCTIVITY

In [3]:
# Set environment variable
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

global_seed = 0

random.seed(global_seed)
np.random.seed(global_seed)

torch.manual_seed(global_seed)
torch.use_deterministic_algorithms(True)

In [4]:
def make_config(years: List[int], state: str, state_ansi: str, fips: str, crop_type: str, grow_season: List[int]):

    config = {
        "FIPS": fips,
        "years": years,
        "state": state.upper(),
        "crop_type": crop_type,
        "data": {
            "HRRR": {
                "short_term": []
            },
            "USDA": [],
            "sentinel": []
        }
    }
    
    for year in years:
        # HRRR data
        hrrr_files = [
            f"HRRR/{year}/{state.upper()}/HRRR_{state_ansi}_{state.upper()}_{year}-{month:02d}.csv"
            for month in range(grow_season[0], grow_season[1] + 1)
        ]
        config["data"]["HRRR"]["short_term"].append(hrrr_files)
        
        # USDA data
        if crop_type=="Soybeans":
            config["data"]["USDA"].append(f"USDA/{crop_type}/{year}/USDA_Soybean_County_{year}.csv")
        else:
            config["data"]["USDA"].append(f"USDA/{crop_type}/{year}/USDA_{crop_type}_County_{year}.csv")
        
        # Sentinel data
        quarters = [
            (f"{year}-01-01", f"{year}-03-31"),
            (f"{year}-04-01", f"{year}-06-30"),
            (f"{year}-07-01", f"{year}-09-30"),
            (f"{year}-10-01", f"{year}-12-31")
        ]
        
        sentinel_files = []
        for start, end in quarters:
            quarter_start = datetime.strptime(start, "%Y-%m-%d")
            quarter_end = datetime.strptime(end, "%Y-%m-%d")
            if (grow_season[0] <= quarter_start.month <= grow_season[1]) or \
               (grow_season[0] <= quarter_end.month <= grow_season[1]):
                sentinel_files.append(f"AG/{state.upper()}/{year}/Agriculture_{state_ansi}_{state.upper()}_{start}_{end}.h5")
        
        config["data"]["sentinel"].append(sentinel_files)
    
    return config

# Train
years = list(range(2018,2022))
state = "AL"
state_ansi = "01"
fips = ['01003', '01015', '01019', '01031', '01039', '01045', '01047', '01053', '01061', 
        '01067', '01069', '01077', '01079', '01083', '01089', '01097', '01099', '01117'] 

crop_type = "Cotton"
grow_season = [4, 9]  # April to September

train_config = make_config(years, state, state_ansi, fips, crop_type, grow_season)
with open('train_config.json', 'w') as file:
    json.dump(train_config, file)

# Test
years = [2022]
test_config = make_config(years,  state, state_ansi, fips, crop_type, grow_season)
with open('test_config.json', 'w') as file:
    json.dump(test_config, file)

## DATA LOADER

In [5]:
class Sentinel2Imagery(Dataset):
    def __init__(self, base_dir, config_file, transform=None):
        self.transform = transform
        self.base_dir = base_dir
        
        with open(config_file, 'r') as f:
            obj = json.load(f)
        
        self.fips_codes = obj["FIPS"]
        self.years = obj["years"]
        self.file_paths = obj["data"]["sentinel"]
    
    def __len__(self):
        return len(self.fips_codes) * len(self.years)

    def __getitem__(self, index):
        fips_index = index // len(self.years)
        year_index = index % len(self.years)
        
        fips_code = self.fips_codes[fips_index]
        year = self.years[year_index]
        file_paths = self.file_paths[year_index]
        
        temporal_list = []
        for file_path in file_paths:
            with h5py.File(os.path.join(self.base_dir, file_path), 'r') as hf:
                groups = hf[fips_code]
                for d in groups.keys():
                    grids = groups[d]["data"]
                    grids = torch.from_numpy(np.asarray(grids))
                    temporal_list.append(grids)
                hf.close()
        x = torch.stack(temporal_list)
        x = x.to(torch.float32)
        x = rearrange(x, 't g h w c -> t g c h w')
        if self.transform:
            t, g, _, _, _ = x.shape
            x = rearrange(x, 't g c h w -> (t g) c h w')
            x = self.transform(x)
            x = rearrange(x, '(t g) c h w -> t g c h w', t=t, g=g)
        return x, fips_code, year

class HRRRComputedDataset(Dataset):
    def __init__(self, base_dir, config_file, column_names=None):
        self.base_dir = base_dir
        self.day_range = [i + 1 for i in range(28)]
        
        with open(config_file, 'r') as f:
            obj = json.load(f)
        
        self.fips_codes = obj["FIPS"]
        self.years = obj["years"]
        self.short_term_file_path = obj["data"]["HRRR"]["short_term"]
        
        if column_names:
            self.column_names = column_names
        else:
            self.column_names = [
                'Avg Temperature (K)', 'Max Temperature (K)', 'Min Temperature (K)',
                'Precipitation (kg m**-2)', 'Relative Humidity (%)', 'Wind Gust (m s**-1)',
                'Wind Speed (m s**-1)', 'Downward Shortwave Radiation Flux (W m**-2)',
                'Vapor Pressure Deficit (kPa)'
            ]

    def __len__(self):
        return len(self.fips_codes) * len(self.years)

    def __getitem__(self, index):
        fips_index = index // len(self.years)
        year_index = index % len(self.years)
        
        fips_code = self.fips_codes[fips_index]
        year = self.years[year_index]
        short_term_file_paths = self.short_term_file_path[year_index]
        x_short = self.get_short_term_val(fips_code, short_term_file_paths)
        x_short = x_short.to(torch.float32)
        return x_short, fips_code, year

    def get_short_term_val(self, fips_code, file_paths):
        df_list = []
        for file_path in file_paths:
            tmp_df = pd.read_csv(os.path.join(self.base_dir, file_path))
            df_list.append(tmp_df)

        df = pd.concat(df_list, ignore_index=True)
        df["FIPS Code"] = df["FIPS Code"].astype(str).str.zfill(5)
        df = df[(df["FIPS Code"] == fips_code) & (df["Daily/Monthly"] == "Daily")]
        df.columns = df.columns.str.strip()

        group_month = df.groupby(['Month'])

        temporal_list = []
        for month, df_month in group_month:
            group_grid = df_month.groupby(['Grid Index'])

            time_series = []
            for grid, df_grid in group_grid:
                df_grid = df_grid.sort_values(by=['Day'], ascending=[True], na_position='first')
                df_grid = df_grid[df_grid.Day.isin(self.day_range)]
                df_grid = df_grid[self.column_names]
                val = self.signed_log_transform(torch.from_numpy(df_grid.values))
                time_series.append(val)

            temporal_list.append(torch.stack(time_series))

        x_short = torch.stack(temporal_list)
        x_short = rearrange(x_short, 'm g d p -> m d g p')
        return x_short

    def signed_log_transform(self, data):
        epsilon = 1e-9  # small constant to avoid log(0)
        return torch.sign(data) * torch.log10(torch.abs(data) + epsilon)

class USDACropDataset(Dataset):
    def __init__(self, base_dir, config_file, crop_type):
        self.base_dir = base_dir
        self.crop_type = crop_type
        
        with open(config_file, 'r') as f:
            obj = json.load(f)
        
        self.fips_codes = obj["FIPS"]
        self.years = obj["years"]
        self.file_paths = obj["data"]["USDA"]

        if crop_type == "Cotton":
            self.column_names = ['PRODUCTION, MEASURED IN 480 LB BALES', 'YIELD, MEASURED IN LB / ACRE']
        else:
            self.column_names = ['PRODUCTION, MEASURED IN BU', 'YIELD, MEASURED IN BU / ACRE']

        
    def __len__(self):
        return len(self.fips_codes) * len(self.years)
    def get_num_classes(self):
        return len(self.fips_encoder.classes_)
    def __getitem__(self, index):
        fips_index = index // len(self.years)
        year_index = index % len(self.years)
        
        fips_code = self.fips_codes[fips_index]
        year = self.years[year_index]
        file_path = self.file_paths[year_index]
        df = pd.read_csv(os.path.join(self.base_dir, file_path))

        df['state_ansi'] = df['state_ansi'].astype(str).str.zfill(2)
        df['county_ansi'] = df['county_ansi'].astype(str).str.zfill(3)

        df = df[(df["state_ansi"] == fips_code[:2]) & (df["county_ansi"] == fips_code[-3:])]

        df = df[self.column_names]
        x = torch.from_numpy(df.values)
        x = x.to(torch.float32)
        x = torch.log(torch.flatten(x, start_dim=0))
        return x, fips_code, year

## MODAL ARCHITECTURE

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange

class SpatialEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        # CNN layers for spatial feature extraction
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        # Global average pooling to handle variable spatial dimensions
        self.gap = nn.AdaptiveAvgPool2d(1)
        
    def forward(self, x):
        # x shape: [batch*t*g, c, h, w]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.gap(x)  # [batch*t*g, 64, 1, 1]
        return x.squeeze(-1).squeeze(-1)  # [batch*t*g, 64]

class TemporalEncoder(nn.Module):
    def __init__(self, input_size=64):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=128,
            num_layers=2,
            batch_first=True,
            dropout=0.2
        )
        
    def forward(self, x):
        # x shape: [batch, t, g, features]
        batch, t, g, features = x.shape
        
        # Combine batch and grid dimensions for LSTM
        x = rearrange(x, 'b t g f -> (b g) t f')
        
        # Run through LSTM
        output, (hidden, _) = self.lstm(x)
        
        # Take final hidden state
        hidden = hidden[-1]  # [batch*g, hidden_size]
        
        # Reshape back to separate batch and grid dimensions
        hidden = hidden.view(batch, g, -1)  # [batch, g, hidden_size]
        
        # Average across grids
        hidden = torch.mean(hidden, dim=1)  # [batch, hidden_size]
        
        return hidden

class CropYieldModel(nn.Module):
    def __init__(self, num_fips):
        super().__init__()
        self.spatial_encoder = SpatialEncoder()
        self.temporal_encoder = TemporalEncoder()
        
        self.fc = nn.Sequential(
            nn.Linear(128 + num_fips, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 2)
        )

        
    def forward(self, x, fips_onehot):
        # x shape: [batch, t, g, c, h, w]
        batch, t, g, c, h, w = x.shape
        
        # Reshape for CNN
        x = rearrange(x, 'b t g c h w -> (b t g) c h w')
        
        # Extract spatial features
        x = self.spatial_encoder(x)  # [batch*t*g, 64]
        
        # Reshape for LSTM
        x = x.view(batch, t, g, -1)  # [batch, t, g, 64]
        
        # Process temporal features
        x = self.temporal_encoder(x)  # [batch, 128]
        
        # Concatenate with FIPS one-hot encoding
        x = torch.cat([x, fips_onehot], dim=1)
        
        # Final prediction
        
        return self.fc(x)

## TRAIN AND TEST

In [7]:
def train_model(model, train_data, epochs=100, patience=5, save_path="best_model.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    model = model.to(device)

    train_sentinel_loader, train_hrrr_loader, train_usda_loader = train_data
    
    criterion = nn.HuberLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    best_train_loss = float('inf')
    counter = 0
    train_losses = []

    for epoch in range(epochs):
        # Training
        model.train()
        train_running_loss = 0.0
        for (sentinel, fips_code, year), (hrrr, _, _), (usda, _, _) in tqdm(zip(train_sentinel_loader, train_hrrr_loader, train_usda_loader), desc=f"Epoch {epoch+1} Training"):
            gc.collect()
            torch.cuda.empty_cache()
            model.half()
            sentinel, hrrr, usda = sentinel.to(device), hrrr.to(device), usda.to(device)
            fips_onehot = torch.tensor(fips_encoder.transform(np.array(fips_code).reshape(-1, 1)), dtype=torch.float32).to(device)      
            optimizer.zero_grad()
            with autocast(device_type=str(device)):
                output = model(sentinel, fips_onehot)
                loss = criterion(output, usda)
            
            loss.backward()
            model.float()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#             torch.autograd.set_detect_anomaly(True)
            optimizer.step()
            train_running_loss += loss.item()
        
        train_epoch_loss = train_running_loss / len(train_sentinel_loader)
        train_losses.append(train_epoch_loss)
    
        scheduler.step()
        
        print(f"Epoch {epoch+1} - Train Loss: {train_epoch_loss:.4f}")
        
        if train_epoch_loss < best_train_loss:
            best_train_loss = train_epoch_loss
            torch.save(model.state_dict(), save_path)
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping")
                break
    return model

def evaluate_model(model, test_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    sentinel_loader, hrrr_loader, usda_loader = test_data
    all_predictions = []
    all_ground_truth = []

    with torch.no_grad():
        for (sentinel, fips_code, year), (hrrr, fips_code, year), (usda, fips_code, year) in zip(sentinel_loader, hrrr_loader, usda_loader):
            sentinel, hrrr, usda = sentinel.to(device), hrrr.to(device), usda.to(device)
            fips_onehot = torch.tensor(fips_encoder.transform(np.array(fips_code).reshape(-1, 1)), dtype=torch.float32).to(device)
            output = model(sentinel, fips_onehot)
            all_predictions.append(output.cpu().numpy())
            all_ground_truth.append(usda.cpu().numpy())
        
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_ground_truth = np.concatenate(all_ground_truth, axis=0)
    print(all_predictions[:,0])
    print(all_ground_truth[:,0])
    print(all_predictions[:,1])
    print(all_ground_truth[:,1])
    results = {}
    for i, metric_name in enumerate(["Production", "Yield"]):
        y_true = torch.from_numpy(all_ground_truth[:, i])
        y_pred = torch.from_numpy(all_predictions[:, i])
        mae = torch.abs(y_true - y_pred).mean()
        mse = ((y_pred - y_true) ** 2).mean()
        rmse = torch.sqrt(mse)
        mape = (torch.abs(y_true - y_pred) / torch.abs(y_true)).mean() * 100
        smape = 100 * (torch.abs(y_true - y_pred) / ((torch.abs(y_true) + torch.abs(y_pred)) / 2)).mean()
        max_error = torch.abs(y_true - y_pred).max()
        corr = torch.corrcoef(torch.stack((y_pred, y_true)))
        metrics = {
            'MAE': round(mae.item(), 2),
            'MSE': round(mse.item(), 2),
            'RMSE': round(rmse.item(), 2),
            'MAPE': round(mape.item(), 2),
            'SMAPE': round(smape.item(), 2),
            'Max Error': round(max_error.item(), 2),
            'Correlation Coefficient': round(corr[0, 1].item(), 2)
        }
        results[metric_name] = metrics
    return results

## Cotton

In [8]:
gc.collect()
torch.cuda.empty_cache()
base_dir = "/kaggle/input/cropnetv2"
train_config = "/kaggle/working/train_config.json"
test_config = "/kaggle/working/test_config.json"

train_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,train_config),batch_size = 1)
train_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,train_config),batch_size = 1)
train_usda_loader = DataLoader(USDACropDataset(base_dir,train_config,crop_type),batch_size = 1)
    
test_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,test_config),batch_size = 1)
test_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,test_config),batch_size = 1)
test_usda_loader = DataLoader(USDACropDataset(base_dir,test_config,crop_type),batch_size = 1)

with open(train_config, 'r') as f:
    obj = json.load(f)
    fips_codes = sorted(obj["FIPS"])
    
fips_encoder = OneHotEncoder(sparse_output=False)
fips_encoder.fit(np.array(fips).reshape(-1, 1))

model = CropYieldModel(len(fips_codes))

model = train_model(model, (train_sentinel_loader, train_hrrr_loader, train_usda_loader),
                    epochs=20, patience=5, save_path="best_model.pth")

results = evaluate_model(model, (test_sentinel_loader, test_hrrr_loader, test_usda_loader))
print(crop_type)
pprint(results)

Using device: cuda


Epoch 1 Training: 72it [03:25,  2.86s/it]


Epoch 1 - Train Loss: 3.2629


Epoch 2 Training: 72it [03:06,  2.59s/it]


Epoch 2 - Train Loss: 0.6955


...

Epoch 14 - Train Loss: 0.5901


Epoch 15 Training: 72it [03:06,  2.59s/it]


Epoch 15 - Train Loss: 0.7183


Epoch 16 Training: 72it [03:05,  2.58s/it]


Epoch 16 - Train Loss: 0.6086
Early stopping
[10.089026 10.075547 10.302738 10.142555 10.218484 10.172125 10.168705
 10.301669 10.17576  10.196869 10.165039 10.240346 10.101841 10.08524
 10.244759  9.999657 10.120121 10.012542]
[ 9.69892    9.230143  10.9937315 10.154246  10.370361   9.792556
 10.122623  10.518673  10.781037  10.4399805 10.868568  10.526749
 10.39513   10.721063  11.098924   9.400961  10.003333   8.9733515]
[6.534198  6.5275645 6.668336  6.5657845 6.6078224 6.586041  6.569924
 6.668332  6.5632615 6.5967817 6.569629  6.6342454 6.5204563 6.5207424
 6.614581  6.476229  6.5427217 6.477821 ]
[6.6618547 7.075809  7.044905  6.605298  6.851185  6.6515718 6.793466
 6.788972  6.7007313 6.7007313 6.6253924 7.0012455 6.980076  6.9966817
 7.1147695 6.555357  6.568078  6.809039 ]
Cotton
{'Production': {'Correlation Coefficient': 0.74,
                'MAE': 0.45,
                'MAPE': 4.45,
                'MSE': 0.29,
                'Max Error': 1.04,
                'RMSE': 0.5

## Soybeans

In [9]:
# Train
years = list(range(2018,2022))
state = "IL"
state_ansi = "17"
fips = ['17005', '17007', '17009', '17011', '17015', '17019', '17025', '17027', '17037', 
        '17045', '17049', '17053', '17055', '17057', '17059', '17063', '17073', '17075', '17077', 
        '17081', '17089', '17091', '17095', '17101', '17103', '17105', '17113', '17115', '17117', 
        '17119', '17121', '17129', '17133', '17139', '17141', '17143', '17145', '17153', '17157', 
        '17163', '17167', '17173', '17177', '17179', '17189', '17193', '17197', '17201', '17203']

crop_type = "Soybeans"
grow_season = [4, 9]  # April to September

train_config = make_config(years, state, state_ansi, fips, crop_type, grow_season)
with open('train_config.json', 'w') as file:
    json.dump(train_config, file)

# Test
years = [2022]
test_config = make_config(years,  state, state_ansi, fips, crop_type, grow_season)
with open('test_config.json', 'w') as file:
    json.dump(test_config, file)

In [10]:
gc.collect()
torch.cuda.empty_cache()
base_dir = "/kaggle/input/cropnetv2"
train_config = "/kaggle/working/train_config.json"
test_config = "/kaggle/working/test_config.json"

train_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,train_config),batch_size = 1)
train_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,train_config),batch_size = 1)
train_usda_loader = DataLoader(USDACropDataset(base_dir,train_config,crop_type),batch_size = 1)
    
test_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,test_config),batch_size = 1)
test_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,test_config),batch_size = 1)
test_usda_loader = DataLoader(USDACropDataset(base_dir,test_config,crop_type),batch_size = 1)

with open(train_config, 'r') as f:
    obj = json.load(f)
    fips_codes = sorted(obj["FIPS"])
    
fips_encoder = OneHotEncoder(sparse_output=False)
fips_encoder.fit(np.array(fips).reshape(-1, 1))

model = CropYieldModel(len(fips_codes))

model = train_model(model, (train_sentinel_loader, train_hrrr_loader, train_usda_loader),
                    epochs=20, patience=5, save_path="best_model.pth")

results = evaluate_model(model, (test_sentinel_loader, test_hrrr_loader, test_usda_loader))
print(crop_type)
pprint(results)

Using device: cuda


Epoch 1 Training: 196it [07:18,  2.24s/it]


Epoch 1 - Train Loss: 3.0597


Epoch 2 Training: 196it [06:49,  2.09s/it]


Epoch 2 - Train Loss: 1.1047


...

Epoch 9 - Train Loss: 0.9595


Epoch 10 Training: 196it [06:53,  2.11s/it]


Epoch 10 - Train Loss: 0.9160


Epoch 11 Training: 196it [06:54,  2.12s/it]


Epoch 11 - Train Loss: 1.0931
Early stopping
[14.842931  14.7063465 14.676423  14.815689  14.764834  14.888553
 14.684739  14.73956   14.823714  14.830167  14.778219  14.757054
 14.706213  14.728454  14.69974   14.738306  14.812417  14.860637
 14.746842  14.771118  14.692784  14.784295  14.838031  14.745439
 14.762967  14.811901  14.855961  14.69806   14.799323  14.718449
 14.716773  14.738996  14.68303   14.76261   14.74095   14.729477
 14.682932  14.718153  14.807362  14.741618  14.84672   14.75712
 14.750951  14.685553  14.764732  14.714651  14.76456   14.66651
 14.75534  ]
[15.402089  14.958139  14.715672  16.273218  15.050401  16.715998
 15.638446  15.553165  15.956517  16.225693  15.585365  16.006493
 15.211013  15.891822  15.290045  15.627236  16.31842   16.762262
 15.36009   15.445339  15.150512  15.927903  16.007277  15.468008
 16.058239  16.700703  16.86885   16.122486  16.281658  15.761849
 15.542731  15.364137  15.346987  15.488862  15.985821  15.533624
 15.245583  14.69181

## Corn

In [11]:
# Train
years = list(range(2018,2022))
state = "IL"
state_ansi = "17"
fips = ['17007', '17011', '17015', '17017', '17019', '17021', '17025', '17027',  
        '17037', '17049', '17053', '17055', '17057', '17059', '17061', '17063', '17073', 
        '17075', '17077', '17081', '17085', '17089', '17093', '17095', '17101', '17103', 
        '17105', '17107', '17113', '17115', '17117', '17119', '17121', '17123', '17133', 
        '17135', '17139', '17141', '17143', '17147', '17157', '17163', '17167', '17169', 
        '17173', '17175', '17177', '17179', '17189', '17193', '17195', '17201', '17203'] 
crop_type = "Corn"
grow_season = [4, 9]  # April to September

train_config = make_config(years, state, state_ansi, fips, crop_type, grow_season)
with open('train_config.json', 'w') as file:
    json.dump(train_config, file)
print("Train config")
pprint(train_config)

# Test
years = [2022]
test_config = make_config(years,  state, state_ansi, fips, crop_type, grow_season)
with open('test_config.json', 'w') as file:
    json.dump(test_config, file)
print("Test config")
pprint(test_config)

Train config
{'FIPS': ['17007',
          '17011',
          '17015',
          '17017',
          '17019',
          '17021',
          '17025',
          '17027',
          '17037',
          '17049',
          '17053',
          '17055',
          '17057',
          '17059',
          '17061',
          '17063',
          '17073',
          '17075',
          '17077',
          '17081',
          '17085',
          '17089',
          '17093',
          '17095',
          '17101',
          '17103',
          '17105',
          '17107',
          '17113',
          '17115',
          '17117',
          '17119',
          '17121',
          '17123',
          '17133',
          '17135',
          '17139',
          '17141',
          '17143',
          '17147',
          '17157',
          '17163',
          '17167',
          '17169',
          '17173',
          '17175',
          '17177',
          '17179',
          '17189',
          '17193',
          '17195',
          '17201',

In [12]:
gc.collect()
torch.cuda.empty_cache()
base_dir = "/kaggle/input/cropnetv2"
train_config = "/kaggle/working/train_config.json"
test_config = "/kaggle/working/test_config.json"

train_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,train_config),batch_size = 1)
train_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,train_config),batch_size = 1)
train_usda_loader = DataLoader(USDACropDataset(base_dir,train_config,crop_type),batch_size = 1)
    
test_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,test_config),batch_size = 1)
test_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,test_config),batch_size = 1)
test_usda_loader = DataLoader(USDACropDataset(base_dir,test_config,crop_type),batch_size = 1)

with open(train_config, 'r') as f:
    obj = json.load(f)
    fips_codes = sorted(obj["FIPS"])
    
fips_encoder = OneHotEncoder(sparse_output=False)
fips_encoder.fit(np.array(fips).reshape(-1, 1))

model = CropYieldModel(len(fips_codes))

model = train_model(model, (train_sentinel_loader, train_hrrr_loader, train_usda_loader),
                    epochs=20, patience=5, save_path="best_model.pth")

results = evaluate_model(model, (test_sentinel_loader, test_hrrr_loader, test_usda_loader))
print(crop_type)
pprint(results)

Using device: cuda


Epoch 1 Training: 212it [07:23,  2.09s/it]


Epoch 1 - Train Loss: 2.9330


Epoch 2 Training: 212it [07:19,  2.07s/it]


Epoch 2 - Train Loss: 1.1144


...

Epoch 18 - Train Loss: 0.8391


Epoch 19 Training: 212it [07:19,  2.07s/it]


Epoch 19 - Train Loss: 0.8117


Epoch 20 Training: 212it [07:21,  2.08s/it]


Epoch 20 - Train Loss: 0.8153
[16.929138 17.28495  17.16301  17.034496 17.196236 17.163212 17.007309
 16.934254 17.196064 17.005657 17.045326 16.802578 17.261654 16.922016
 17.06503  17.058468 17.219997 17.270031 16.778854 16.755219 17.075283
 17.271183 17.136593 17.30402  16.916214 17.27752  17.263279 16.990482
 17.305828 17.04565  17.130127 16.8539   16.892147 17.010498 16.737762
 17.119053 17.076347 17.288216 17.126019 17.050762 17.058334 17.035717
 17.31791  16.803612 17.213993 17.038155 17.18021  17.189415 16.945545
 16.877167 17.148407 17.017467 17.001244]
[16.461685 17.813234 17.14446  16.6308   17.875126 17.532614 16.464306
 16.747557 17.526592 16.709764 17.217207 15.608103 17.13499  16.099014
 16.859509 16.810692 17.720045 17.986538 15.473167 15.850085 16.730358
 16.503767 16.54127  17.340046 16.363    17.73284  17.939285 17.510695
 18.077923 17.29758  17.347853 16.811243 16.517073 16.807787 15.867195
 17.310411 16.688057 17.65936  16.922785 17.099276 16.115091 16.610472
 17.5

## Winter Wheat

In [13]:
def make_config(years: List[int], state: str, state_ansi: str, fips: str, crop_type: str, grow_season: List[int]):
    """
    Creates configuration for winter wheat data collection
    grow_season: List containing [start_month, end_month] of the growing cycle
                 For winter wheat, this spans across year boundary
    """
    config = {
        "FIPS": fips,
        "years": years,
        "state": state.upper(),
        "crop_type": crop_type,
        "data": {
            "HRRR": {
                "short_term": []
            },
            "USDA": [],
            "sentinel": []
        }
    }
   
    for year in years:
        # HRRR data - need to consider months from previous year's fall
        hrrr_files = []
        # Previous year's fall months (planting)
        for month in range(9, 13):  # September to December
            hrrr_files.append(f"HRRR/{year-1}/{state.upper()}/HRRR_{state_ansi}_{state.upper()}_{year-1}-{month:02d}.csv")
        # Current year's winter and spring months (growing and harvest)
        for month in range(1, 7):  # January to July
            hrrr_files.append(f"HRRR/{year}/{state.upper()}/HRRR_{state_ansi}_{state.upper()}_{year}-{month:02d}.csv")
        
        config["data"]["HRRR"]["short_term"].append(hrrr_files)
       
        # USDA data
        config["data"]["USDA"].append(f"USDA/{crop_type}/{year}/USDA_WinterWheat_County_{year}.csv")
       
        # Sentinel data - need to cover previous fall to current summer
        quarters = [
            # Previous year quarters
            (f"{year-1}-10-01", f"{year-1}-12-31"),  # Q4 (planting)
            # Current year quarters
            (f"{year}-01-01", f"{year}-03-31"),      # Q1 (winter growth)
            (f"{year}-04-01", f"{year}-06-30"),      # Q2 (spring growth)
        ]
       
        sentinel_files = []
        for start, end in quarters:
            sentinel_files.append(f"AG/{state.upper()}/{start[:4]}/Agriculture_{state_ansi}_{state.upper()}_{start}_{end}.h5")
       
        config["data"]["sentinel"].append(sentinel_files)
   
    return config

# Train
years = list(range(2018, 2022))
state = "IL"
state_ansi = "17"
fips = ['17011', '17013', '17023', '17025', '17027', '17037', '17047', '17049', '17067', 
        '17083', '17089', '17095', '17119', '17121', '17125', '17133', '17141', '17157', '17159', 
        '17163', '17173', '17177', '17179', '17189', '17201'] 
crop_type = "WinterWheat"
grow_season = [9, 6]  # September to July (spanning across years)

train_config = make_config(years, state, state_ansi, fips, crop_type, grow_season)
with open('train_config.json', 'w') as file:
    json.dump(train_config, file)


# Test
years = [2022]
test_config = make_config(years, state, state_ansi, fips, crop_type, grow_season)
with open('test_config.json', 'w') as file:
    json.dump(test_config, file)


In [14]:
gc.collect()
torch.cuda.empty_cache()
base_dir = "/kaggle/input/cropnetv2"
train_config = "/kaggle/working/train_config.json"
test_config = "/kaggle/working/test_config.json"

train_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,train_config),batch_size = 1)
train_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,train_config),batch_size = 1)
train_usda_loader = DataLoader(USDACropDataset(base_dir,train_config,crop_type),batch_size = 1)
    
test_sentinel_loader = DataLoader(Sentinel2Imagery(base_dir,test_config),batch_size = 1)
test_hrrr_loader = DataLoader(HRRRComputedDataset(base_dir,test_config),batch_size = 1)
test_usda_loader = DataLoader(USDACropDataset(base_dir,test_config,crop_type),batch_size = 1)

with open(train_config, 'r') as f:
    obj = json.load(f)
    fips_codes = sorted(obj["FIPS"])
    
fips_encoder = OneHotEncoder(sparse_output=False)
fips_encoder.fit(np.array(fips).reshape(-1, 1))

model = CropYieldModel(len(fips_codes))

model = train_model(model, (train_sentinel_loader, train_hrrr_loader, train_usda_loader),
                    epochs=20, patience=5, save_path="best_model.pth")

results = evaluate_model(model, (test_sentinel_loader, test_hrrr_loader, test_usda_loader))
print(crop_type)
pprint(results)

Using device: cuda


Epoch 1 Training: 100it [05:44,  3.45s/it]


Epoch 1 - Train Loss: 4.2915


Epoch 2 Training: 100it [05:25,  3.26s/it]


Epoch 2 - Train Loss: 1.1492


...

Epoch 13 - Train Loss: 0.8326


Epoch 14 Training: 100it [05:21,  3.22s/it]


Epoch 14 - Train Loss: 0.9601


Epoch 15 Training: 100it [05:22,  3.22s/it]


Epoch 15 - Train Loss: 0.8291
Early stopping
[12.065768  12.002904  12.070011  12.105804  12.138816  12.039427
 12.078158  12.10844   12.052681  12.0421295 11.947662  11.972492
 12.207968  12.187806  12.031501  12.139296  12.073464  12.183838
 12.111438  12.136925  11.985008  11.930388  12.035559  12.192896
 11.991358 ]
[11.91839   12.106253  12.628067  14.008607  14.616166  12.966879
 12.694653  13.144125  11.877568  12.524527  12.264341  10.687389
 13.946539  13.949167  11.827736  14.346726  12.863593  14.757859
 13.3984785 14.402185  13.153862  12.396693  12.621488  15.340044
 12.464583 ]
[3.984835  3.945261  3.9563148 3.9726503 3.9818919 3.9652646 3.9719622
 3.9916441 3.973873  3.9607239 3.9291275 3.932956  4.012988  4.0066986
 3.9450736 3.9808257 3.9784133 4.0085807 3.9848819 3.9863217 3.9298685
 3.9145608 3.9545815 3.9867175 3.9343486]
[4.572647  4.352855  4.5325994 4.447346  4.3907385 4.2904596 4.2668962
 4.3605475 4.377014  4.355426  4.506454  4.433195  4.3579903 4.4236484
 4.1